In [ ]:
from joblib import load
from tensorflow.keras.models import load_model

model1 = load_model("model_with_two_dataset.h5")  


def extract_label_from_filename(path):
    filename = os.path.basename(path)
    
    if '_' in filename:
        emotion = filename.split('_')[2].split('.')[0].lower()
        if emotion =='ps':
             return 'surprised'
        if emotion =='fearful':
                return 'fear'
        else:
             return emotion
    elif '-' in filename:
        number = filename.split('-')[2]
        emotion_map = {
            '01': 'neutral', '02': 'calm', '03': 'happy', '04': 'sad',
            '05': 'angry', '06': 'fear', '07': 'disgust', '08': 'surprised'
        }
        return emotion_map.get(number)

In [18]:
import librosa
import numpy as np

def extract_mfcc(file_path):
    y, sr = librosa.load(file_path, sr=16000)
    y, _ = librosa.effects.trim(y)
    y = librosa.util.normalize(y)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)

    if mfcc.shape[1] >= 100:
        mfcc = mfcc[:, :100]
    else:
        pad_width = 100 - mfcc.shape[1]
        mfcc = np.pad(mfcc, ((0, 0), (0, pad_width)), mode='constant')

    return mfcc


In [19]:
from sklearn.preprocessing import LabelEncoder


def get_label_encoder():
    duygular = [
    'angry',
    'calm',
    'disgust',
    'fear',
    'happy',
    'neutral',
    'sad',
    'surprised'
]
    encoder = LabelEncoder()
    encoder.fit(duygular)
    return encoder

In [20]:
def predict_single_audio(file_path, model, label_encoder):
    true_label = extract_label_from_filename(file_path)
    mfcc = extract_mfcc(file_path)

    mfcc = mfcc[..., np.newaxis]  
    mfcc = np.expand_dims(mfcc, axis=0)

    prediction = model.predict(mfcc)
    predicted_index = np.argmax(prediction)
    predicted_label = label_encoder.inverse_transform([predicted_index])[0]

    print("Dosya Adı:", file_path)
    print("Gerçek Etiket:", true_label)
    print("Model Tahmini:", predicted_label)


In [21]:
encoder =get_label_encoder()
predict_single_audio("ses/ses_dosyalarim_tess/TESS Toronto emotional speech set data/OAF_angry/OAF_back_angry.wav", model1,encoder)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 495ms/step
Dosya Adı: ses/ses_dosyalarim_tess/TESS Toronto emotional speech set data/OAF_angry/OAF_back_angry.wav
Gerçek Etiket: tess/tess toronto emotional speech set data/oaf
Model Tahmini: angry
